# Module 1: Preprocessing Features  

This module focuses on processing single-cell image-based profiles into **aggregate profiles**. The aggregation is performed at two distinct levels:  

1. **Replicate Level**: Profiles are aggregated for individual replicates.  
2. **Mechanism of Action (MOA) Level**: Profiles are further aggregated based on shared mechanisms of action.  

To enable these aggregation steps, certain augmentations to the plate metadata are required. These augmentations ensure the metadata aligns appropriately with the profiles for accurate processing.

---

### Profile Descriptions:
- **Aggregate Profiles at the Replicate Level**: Profiles generated by combining data for individual replicates within the same condition or treatment group.  
- **Aggregate Profiles at the Mechanism of Action (MOA) Level**: Profiles generated by combining replicates that share a common mechanism of action, allowing downstream analysis of MOA-specific features.

In [1]:
import sys
import pathlib

import pandas as pd
from pycytominer import aggregate

# importing analysis modules
sys.path.append("../../")
from src import io_utils, data_utils

Setting up parameters and paths

In [2]:
# setting up parameters
moa_tag = "with_moa"

In [3]:
# setting up path
data_dir = pathlib.Path("../data").resolve(strict=True)
metadata_dir = (data_dir / "metadata").resolve(strict=True)

# set bar code path
barcode_path = (metadata_dir / "barcode_platemap.csv").resolve(strict=True)

# set pathway metdata path
pathway_metadata_path = (metadata_dir / "pathways_platemap.csv").resolve(strict=True)

# getting all platemaps files
all_platemap_paths = list(metadata_dir.glob("Target_Selective_Library_Screen_*.csv"))

# setting up results directory
results_dir = pathlib.Path("./results").resolve()
results_dir.mkdir(exist_ok=True)

# making a results dir for the augmented platemaps with moa
results_platemap_dir = (results_dir / "platemaps_w_moa").resolve()
results_platemap_dir.mkdir(exist_ok=True)

# making a results dir for aggregatd profiles
results_moa_agg_profiles_dir = (results_dir / "aggregated_profiles/moa_agg").resolve()
results_repl_agg_profiles_dir = (results_dir / "aggregated_profiles/repl_agg").resolve()
results_moa_agg_profiles_dir.mkdir(exist_ok=True, parents=True)
results_repl_agg_profiles_dir.mkdir(exist_ok=True, parents=True)

# updated barcode out path# Save the updated barcode metadata to a CSV file
updated_barcode_path = (
    results_platemap_dir / f"barcode_platemaps_{moa_tag}.csv"
).resolve()

This process integrates pathway metadata into experimental platemaps to enrich single-cell image-based profiles with additional biological context. First, the pathway metadata, containing information such as treatment identifiers, well positions, and pathways, is filtered for each specific plate. This filtered metadata is then merged with the corresponding platemap, linking treatments and well positions to their associated pathways. The resulting augmented platemaps provide a unified view that connects experimental treatments to mechanisms of action (MOA), enabling downstream analysis and interpretation of pathway-specific effects.

In [4]:
# Load the pathways metadata into a DataFrame
pathway_meta_df = pd.read_csv(pathway_metadata_path)

# Iterate over all platemap files to augment them with pathway MOA information
for platemap_path in all_platemap_paths:
    # Generate an output filename for the augmented platemap
    # Example: Extract prefix and plate ID, then append a "with_moa" tag
    prefix = platemap_path.stem.rsplit("_", 2)[0]
    plate_id = platemap_path.stem.split("_", 4)[-1]
    tag = "with_moa"
    outname = f"{prefix}_{plate_id}_{tag}.csv"

    # Load the current platemap into a DataFrame
    plate_meta_df = pd.read_csv(platemap_path)

    # Filter the pathway metadata to include only rows corresponding to the current plate
    # Select relevant columns: 'UCD ID' (treatment), 'Well' (well position), and 'Pathway' (moa)
    plate_pathway_meta_df = pathway_meta_df.loc[pathway_meta_df["Plate"] == plate_id][
        ["UCD ID", "Well", "Pathway"]
    ]

    # Merge the pathway metadata into the platemap using 'treatment' and 'well_position'
    # as keys. Perform a left join to retain all rows from the platemap
    merged_df = plate_meta_df.merge(
        plate_pathway_meta_df,
        left_on=["treatment", "well_position"],
        right_on=["UCD ID", "Well"],
        how="left",
    )

    # Save the augmented platemap with MOA information to the output directory
    merged_df.to_csv(results_platemap_dir / outname, index=False)

In [5]:
# Load the original barcode metadata
barcodes = io_utils.load_barcodes(barcode_path=barcode_path)

# Initialize a list to store the updated barcode entries
updated_barcodes = []

# Iterate through each batch and its associated plates in the barcode metadata
for batch_id, plates in barcodes.items():
    # Generate the updated platemap name with the new tag
    updated_platemap_name = f"{list(plates.keys())[0]}_{moa_tag}"

    # Iterate through all profiles associated with the current batch of plates
    for plate_profiles in plates.values():
        for profile_name in plate_profiles:
            # Append the updated profile-barcode mapping
            updated_barcodes.append([profile_name, updated_platemap_name])

# Convert the updated barcode list into a DataFrame
updated_barcodes_df = pd.DataFrame(
    updated_barcodes, columns=["plate_barcode", "platemap_file"]
)

updated_barcodes_df.to_csv(updated_barcode_path, index=False)
updated_barcodes_df

,plate_barcode,platemap_file
0,localhost240926150001,Target_Selective_Library_Screen_Plate_1_with_moa
1,localhost240927060001,Target_Selective_Library_Screen_Plate_1_with_moa
2,localhost240927120001,Target_Selective_Library_Screen_Plate_1_with_moa
3,localhost240928120001,Target_Selective_Library_Screen_Plate_1_with_moa


Next, we aim to **annotate the single-cell profiles** by integrating them with the newly augmented platemaps. This process begins by loading the updated barcode metadata, which now includes MOA information. Using these barcodes, batch profiles are loaded from the specified data directory. The batch profiles are then annotated with additional metadata by aligning the platemap's well positions (`Metadata_well_position`) with the profile metadata (`Image_Metadata_Well`). Finally, the annotated profiles are concatenated at the batch level, combining all plates within each batch into a single unified dataset.

In [6]:
# Load the updated barcode metadata containing MOA information
updated_barcodes = io_utils.load_barcodes(barcode_path=updated_barcode_path)

# Load batch profiles using the updated barcodes
batch_profiles = io_utils.batch_load_profiles(
    path_to_data_dir=data_dir, barcodes=updated_barcodes
)

# Annotate the loaded batch profiles with metadata
annotated_profiles = data_utils.annotate_batch(
    batch_profiles,
    metadata_dir=results_platemap_dir,
    platemap_metadata_query="Metadata_well_position",
    profile_metadata_query="Image_Metadata_Well",
)

# concatenate all plates in a batch into one
concat_batch_profiles = data_utils.concat_batch_profiles(annotated_profiles)

Finally, we **aggregate the single-cell profiles** into two distinct sets:  

1. **Replicate-Level Aggregation**: Profiles are combined at the replicate level, grouping data from individual experimental replicates.  
2. **MOA-Level Aggregation**: Profiles are further aggregated based on their shared **Mechanism of Action (MOA)**, enabling pathway-specific analysis.  

In [7]:
for batch_id, batch_profile in concat_batch_profiles.items():
    # Generate aggregate profiles at the replicate level
    # Group by 'Metadata_plate_id' (plate ID) and 'Metadata_treatment' (treatment/compound)
    replicate_agg_df = aggregate(
        batch_profile, strata=["Metadata_plate_id", "Metadata_treatment"]
    )

    # Generate aggregate profiles at the Mechanism of Action (MOA) level
    # Group by 'Metadata_plate_id' (plate ID) and 'Metadata_Pathway' (MOA annotation)
    moa_agg_df = aggregate(
        batch_profile, strata=["Metadata_plate_id", "Metadata_Pathway"]
    )

    # Save the replicate-level aggregate profiles as a parquet file
    replicate_agg_df.to_parquet(
        results_repl_agg_profiles_dir / f"{batch_id}_repl_aggregate_profiles.parquet"
    )
    
    # Save the MOA-level aggregate profiles as a parquet file
    moa_agg_df.to_parquet(
        results_moa_agg_profiles_dir / f"{batch_id}_moa_aggregate_profiles.parquet"
    )
